In [ ]:
def get_fin(stock):
    import requests
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import pandas as pd
    import time

    url = f"https://www.finnomena.com/stock/{stock}"
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(url) 

    time.sleep(2) 
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    driver.close()

    all_divs = soup.find_all('div', {'class' : "content" })
    n = 0
    quarter = []
    total_asset = []
    total_debt = []
    shareholder_equity = []
    paid_up_capital = []
    total_revenue = []
    net_profit = []
    esp = []
    roa = []
    roe = []
    net_profit_margin = []
    market_capitalization = []
    p_e = []
    p_bv = []
    dividend_yield = []

    buffer = []

    for i in all_divs:
        if n == 0:
            year = i.find_all('div', {'class' : "year" })
            for j in year:
                q = j.text[2:]+j.text[1:2]+j.text[:1]
                quarter.append(q)
            quarter[-1] = quarter[-1][-1:]+quarter[-1][-2:-1]+quarter[-1][:-2]
            n+=1
            continue
        elif n == 1:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(total_asset) < 40:
                    total_asset.append(data)
                elif len(total_debt) < 40:
                    total_debt.append(data)
                elif len(shareholder_equity) < 40:
                    shareholder_equity.append(data)
                elif len(paid_up_capital) < 40:
                    paid_up_capital.append(data)
                elif len(total_revenue) < 40:
                    total_revenue.append(data)
                elif len(buffer) < 80:
                    buffer.append('1')
                elif len(net_profit) < 40:
                    net_profit.append(data)
                elif len(buffer) < 80+160:
                    buffer.append('1')
                elif len(esp) < 40:
                    esp.append(data)
        elif n == 2:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(roa) < 40:
                    roa.append(data)
                elif len(roe) < 40:
                    roe.append(data)
                elif len(buffer) < 80+160+80:
                    buffer.append('1')
                elif len(net_profit_margin) < 40:
                    net_profit_margin.append(data)
        elif n == 3:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(buffer) < 80+160+80+40:
                    buffer.append("1")
                elif len(market_capitalization) < 40:
                    market_capitalization.append(data)
                elif len(p_e) < 40:
                    p_e.append(data)
                elif len(p_bv) < 40:
                    p_bv.append(data)
                elif len(buffer) < 80+160+80+40+40:
                    buffer.append("1")
                elif len(dividend_yield) < 40:
                    dividend_yield.append(data)
        n+=1

    stock_statement = {"quarter":quarter,"total_asset":total_asset,"total_debt":total_debt,"shareholder_equity":shareholder_equity,"paid_up_capital":paid_up_capital,"total_revenue":total_revenue,"net_profit":net_profit,"esp":esp,"roa":roa,"roe":roe,"net_profit_margin":net_profit_margin,"market_capitalization":market_capitalization,"p_e":p_e,"p_bv":p_bv,"dividend_yield":dividend_yield}
    df = pd.DataFrame(stock_statement)

    row_lists = df.apply(lambda row: row.tolist(), axis=1)

    return row_lists.to_list() 


In [ ]:
def insert_financial(stock_id,finance):
    from sqlite3 import connect
    conn = connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO new_financial_statement VALUES (null,{stock_id},'{finance[0]}',{finance[1]},{finance[2]},{finance[3]},{finance[4]},{finance[5]},{finance[6]},{finance[7]},{finance[8]},{finance[9]},{finance[10]},{finance[11]},{finance[12]},{finance[13]},{finance[14]})")
    conn.commit()#commit change to db
    conn.close()#disconnect

In [ ]:
from data.function import Stock,Categories
stock = Categories().get_all_stock()
for i in stock:
    try:
        temp = Stock(i)
        id = temp.get_stock_id()
        financial = get_fin(i)
        for j in financial:
            insert_financial(id,j)
    except:
        pass